In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Dense

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from tensorflow.keras.optimizers import SGD

from sklearn.datasets import load_iris
from tensorflow.keras.datasets.mnist import load_data

In [3]:
# Iris dataset

# Load the dataset (a dictionary) and get the features DataFrame and target values from the dictionary
iris = load_iris(as_frame=True)
iris_df = iris.data
iris_y = iris.target

# Shuffle the features and the target values in the same way
idx = np.random.permutation(iris_df.index)
iris_df.reindex(idx)
iris_y.reindex(idx)
iris_df.reset_index(drop=True, inplace=True)
iris_y.reset_index(drop=True, inplace=True)

# Split off the test set: 20% of the dataset.
dev_iris_df, test_iris_df, dev_iris_y, test_iris_y = train_test_split(iris_df, iris_y, train_size=0.8, 
                                                                              random_state=4)

# Create the preprocessor
iris_preprocessor = ColumnTransformer([
        ("scaler", StandardScaler(), iris_df.columns)],
        remainder="passthrough")

In [4]:
inputs = Input(shape=(4,))
x = Dense(64, activation="relu")(inputs)
x = Dense(64, activation="relu")(x)
outputs = Dense(3, activation="softmax")(x)
iris_model = Sequential(Model(inputs, outputs))

iris_model.compile(optimizer=SGD(learning_rate=0.01), loss="sparse_categorical_crossentropy")

In [5]:
# Create a pipeline
iris_pipeline = Pipeline([
    ("preprocessor", iris_preprocessor),
    ("predictor", KerasClassifier(build_fn=lambda: iris_model, verbose=0, epochs=60, batch_size=32))
])

In [6]:
# Accuracy estimation

iris_pipeline.fit(dev_iris_df, dev_iris_y)

# May get same warning
accuracy_score(test_iris_y, iris_pipeline.predict(test_iris_df))

0.9666666666666667

In [7]:
# Keras has a utility function for downloading it into four Numpy arrays
# To get this to work on macOS, I also had to run this in a terminal:
# $ /Applications/Python\ 3.8/Install\ Certificates.command
# You may need something similar
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = load_data()

11501568/11490434 [==============================] - 2s 0us/step


In [8]:
mnist_x_train = mnist_x_train.reshape((60000, 28 * 28))

mnist_x_test = mnist_x_test.reshape((10000, 28 * 28))

In [9]:
inputs = Input(shape=(28 * 28,))
x = Rescaling(scale=1./255)(inputs)
x = Dense(512, activation="relu")(x)
outputs = Dense(10, activation="softmax")(x)
mnist_model = Model(inputs, outputs)

mnist_model.compile(optimizer=SGD(learning_rate=0.01), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [10]:
mnist_model.fit(mnist_x_train, mnist_y_train, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6024 - accuracy: 0.8533
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3197 - accuracy: 0.9111
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2725 - accuracy: 0.9242: 0s - loss:
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2420 - accuracy: 0.9327
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2194 - accuracy: 0.9388
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2004 - accuracy: 0.9446
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1851 - accuracy: 0.9485
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1717 - accuracy: 0.9519
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1603 - accuracy: 0.9550: 0s
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/st

In [11]:
test_loss, test_acc = mnist_model.evaluate(mnist_x_test, mnist_y_test)
test_acc

313/313 [==============================] - 1s 2ms/step - loss: 0.1477 - accuracy: 0.9575


0.9574999809265137